In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_5_9_5,0.998830,0.997530,0.998095,0.998012,0.000219,0.000309,0.000705,0.000496,"Hidden Size=[15, 7], regularizer=0.01, learnin..."
1,model_5_9_4,0.998826,0.998104,0.998164,0.998213,0.000220,0.000238,0.000680,0.000446,"Hidden Size=[15, 7], regularizer=0.01, learnin..."
2,model_5_9_6,0.998819,0.996949,0.998030,0.997813,0.000221,0.000382,0.000729,0.000546,"Hidden Size=[15, 7], regularizer=0.01, learnin..."
3,model_5_9_3,0.998799,0.998654,0.998236,0.998410,0.000225,0.000169,0.000653,0.000397,"Hidden Size=[15, 7], regularizer=0.01, learnin..."
4,model_5_9_7,0.998796,0.996374,0.997970,0.997617,0.000225,0.000454,0.000752,0.000594,"Hidden Size=[15, 7], regularizer=0.01, learnin..."
...,...,...,...,...,...,...,...,...,...,...
226,model_7_9_1,0.993839,0.996025,0.995458,0.995732,0.001153,0.000384,0.000574,0.000474,"Hidden Size=[25, 15], regularizer=0.01, learni..."
233,model_4_9_5,0.993551,0.987366,0.990463,0.989064,0.001206,0.001578,0.001557,0.001568,"Hidden Size=[15, 5], regularizer=0.1, learning..."
254,model_4_9_4,0.992978,0.989011,0.991759,0.990517,0.001314,0.001373,0.001345,0.001360,"Hidden Size=[15, 5], regularizer=0.1, learning..."
259,model_7_9_0,0.992772,0.997306,0.996387,0.996821,0.001352,0.000260,0.000457,0.000353,"Hidden Size=[25, 15], regularizer=0.01, learni..."
